## 1-Axis tracker example

Example demonstrating Radiance gencumulativesky for 1-axis tracking.

#### Prerequisites (Step 0):
This software requires the previous installation of RADIANCE from https://github.com/NREL/Radiance/releases.

Make sure you add radiance to the system PATH so Python can interact with the radiance program

If you are on a PC you should also copy the Jaloxa radwinexe-5.0.a.8-win64.zip executables into `program files/radiance/bin`: http://www.jaloxa.eu/resources/radiance/radwinexe.shtml

#### STEP 1: Install and import bifacial_radiance

 - clone the bifacial_radiance repo to your local directory
 - navigate to the \bifacial_radiance directory which contains setup
 - run `pip install -e .  `  ( the period . is required, the -e flag is optional and installs in development mode where changes to the bifacial_radiance.py files are immediately incorporated into the module if you re-start the python kernel)

#### STEP 2: Move gencumulativesky.exe
Copy gencumulativesky.exe from the repo's `/bifacial_radiance/data/` directory and copy into your Radiance install directory.
This is typically found in `/program files/radiance/bin/`.  

#### STEP 3: Create a local Radiance directory for storing the scene files created
Keep scene geometry files separate from the bifacial_radiance directory.  Create a local directory somewhere that will be referenced in the next step.

#### STEP 4: Reboot the computer
This makes sure the PATH is updated




In [1]:
#testfolder = r'E:\Documents\Python Scripts\Test1axisFolder'  #point to an empty directory or existing Radiance directory
# tracker geometry options:
module_height = 1.7  # module portrait dimension in meters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.3     # ground albedo
hub_height = 2   # tracker height at 0 tilt in meters (hub height)
limit_angle = 45 # tracker rotation limit angle

In [2]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
import easygui  # this is only required if you want a graphical directory picker.  Note:  easygui sometimes opens in the background forcing you to hunt for the window!  
testfolder = easygui.diropenbox(msg = 'Select or create an empty directory for the Radiance tree',title='Browse for empty Radiance directory')

demo = RadianceObj(path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) #Pull TMY data for any global lat/lon. In this case, Richmond, VA
metdata = demo.readEPW(epwfile) # read in the weather data

# create metdata files for each condition
trackerdict = demo.set1axis(metdata, limit_angle = limit_angle, backtrack = True, gcr = gcr)

# new gencumulativesky function: demo.genCumSky1axis
trackerdict = demo.genCumSky1axis(trackerdict)
# Create a new moduletype: Prism Solar Bi60. width = .984m height = 1.695m. Bifaciality = 0.90
demo.makeModule(name='Prism Solar Bi60',x=0.984,y=module_height)
# print available module types
demo.printModules()


path = C:\Users\cdeline\Documents\Python Scripts\Test1axisFolder
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!


C:\Users\cdeline\Anaconda2\lib\site-packages\pvlib\tracking.py:423: RuntimeWarning: invalid value encountered in minimum
  temp = np.minimum(axes_distance*cosd(wid), 1)


Saving file EPWs\1axis_0.0.csv, # points: 232
Saving file EPWs\1axis_-30.0.csv, # points: 203
Saving file EPWs\1axis_35.0.csv, # points: 97
Saving file EPWs\1axis_5.0.csv, # points: 94
Saving file EPWs\1axis_-25.0.csv, # points: 115
Saving file EPWs\1axis_40.0.csv, # points: 194
Saving file EPWs\1axis_10.0.csv, # points: 173
Saving file EPWs\1axis_-20.0.csv, # points: 232
Saving file EPWs\1axis_45.0.csv, # points: 804
Saving file EPWs\1axis_15.0.csv, # points: 208
Saving file EPWs\1axis_-15.0.csv, # points: 162
Saving file EPWs\1axis_-45.0.csv, # points: 857
Saving file EPWs\1axis_20.0.csv, # points: 113
Saving file EPWs\1axis_-10.0.csv, # points: 94
Saving file EPWs\1axis_-40.0.csv, # points: 77
Saving file EPWs\1axis_25.0.csv, # points: 223
Saving file EPWs\1axis_-5.0.csv, # points: 209
Saving file EPWs\1axis_-35.0.csv, # points: 154
Saving file EPWs\1axis_30.0.csv, # points: 150
message: There were 163 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1

In [3]:
# create a scene using panels in portrait, 2m hub height, 0.33 GCR. NOTE: clearance needs to be calculated at each step. hub height is constant
sceneDict = {'pitch':module_height / gcr,'height':hub_height,'orientation':'portrait'}  
module_type = 'Prism Solar Bi60'
trackerdict = demo.makeScene1axis(trackerdict,module_type,sceneDict, nMods = 20, nRows = 7) #makeScene creates a .rad file with 20 modules per row, 7 rows.

trackerdict = demo.makeOct1axis(trackerdict)
# Now we need to run analysis and combine the results into an annual total.  This can be done by calling scene.frontscan and scene.backscan
trackerdict = demo.analysis1axis(trackerdict)

# the frontscan and backscan include a linescan along a chord of the module, both on the front and back.  
# Return the minimum of the irradiance ratio, and the average of the irradiance ratio along a chord of the module.
print('Annual RADIANCE bifacial ratio for 1-axis tracking: %0.3f - %0.3f' %(min(demo.backRatio), np.mean(demo.backRatio)) )



created 1axis_0.0.oct created 1axis_-30.0.oct created 1axis_35.0.oct created 1axis_5.0.oct created 1axis_-25.0.oct created 1axis_40.0.oct created 1axis_10.0.oct created 1axis_-20.0.oct created 1axis_45.0.oct created 1axis_15.0.oct created 1axis_-15.0.oct created 1axis_-45.0.oct created 1axis_20.0.oct created 1axis_-10.0.oct created 1axis_-40.0.oct created 1axis_25.0.oct created 1axis_-5.0.oct created 1axis_-35.0.oct created 1axis_30.0.oct linescan in process: 1axis_0.0_Front
linescan in process: 1axis_0.0_Back
saved: results\irr_1axis_0.0.csv
linescan in process: 1axis_-30.0_Front
linescan in process: 1axis_-30.0_Back
saved: results\irr_1axis_-30.0.csv
linescan in process: 1axis_35.0_Front
linescan in process: 1axis_35.0_Back
saved: results\irr_1axis_35.0.csv
linescan in process: 1axis_5.0_Front
linescan in process: 1axis_5.0_Back
saved: results\irr_1axis_5.0.csv
linescan in process: 1axis_-25.0_Front
linescan in process: 1axis_-25.0_Back
saved: results\irr_1axis_-25.0.csv
linescan in 

In [ ]:
# Note: same workflow can use stored self inputs rather than passing trackerdict
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
#import easygui  # this is only required if you want a graphical directory picker  
#testfolder = easygui.diropenbox(msg = 'Select or create an empty directory for the Radiance tree',title='Browse for empty Radiance directory')

demo = RadianceObj(path = testfolder)  # Create a RadianceObj 'object'

demo.setGround(0.2) # input albedo number or material name like 'concrete'.  To see options, run this without any input.

epwfile = demo.getEPW(37.5,-77.6) #pull TMY data for any global lat/lon
    
metdata = demo.readEPW(epwfile) # read in the weather data

# create metdata files for each condition
demo.set1axis()

# new gencumulativesky function: demo.genCumSky1axis
demo.genCumSky1axis()
# Create a new moduletype: Prism Solar Bi60. x = .984 y = 1.695. Bifaciality = 0.90
demo.makeModule(name='Prism Solar Bi60',x=0.984,y=1.695)
# print available module types
demo.printModules()
# create a scene using panels in portrait, 2m hub height, 0.33 GCR. NOTE: clearance needs to be calculated at each step. hub height is constant
sceneDict = {'pitch':module_height / gcr,'height':hub_height,'orientation':'portrait'}  
module_type = 'Prism Solar Bi60'
demo.makeScene1axis(moduletype=module_type,sceneDict = sceneDict, nMods = 20, nRows = 7) #makeScene creates a .rad file with 20 modules per row, 7 rows.

demo.makeOct1axis()
# Now we need to run analysis and combine the results into an annual total.  This can be done by calling scene.frontscan and scene.backscan
trackerdict = demo.analysis1axis()

In [ ]:
# the frontscan and backscan include a linescan along a chord of the module, both on the front and back.  
# Return the minimum of the irradiance values, and the average of the irradiance values along a chord of the module.
print('Annual bifacial ratio for 1-axis tracking: %0.3f - %0.3f' %(min(demo.backRatio), np.mean(demo.backRatio)) )

